In [1]:
import pandas as pd
import string, csv, html

from pathlib import Path
from bs4 import BeautifulSoup

In [2]:
papers_file = './data/papers_22072019.csv.xz'
works_file  = './data/works_22072019.csv.xz'

In [3]:
# Artigos Extraidos
df_papers = pd.read_csv(papers_file, sep='|', compression='xz')
df_papers.drop(df_papers.loc[df_papers['idioma']!='Inglês'].index, inplace=True)
df_papers.drop(['id_doc','autores_cnpq','autores',
                'idioma','revista','natureza','ano'], axis=1, inplace=True)
df_papers.dropna(inplace=True)
#df_papers.info()

# Trabalhos Extraidos
df_works = pd.read_csv(works_file, sep='|', compression='xz')
df_works.drop(df_works.loc[df_works['idioma']!='Inglês'].index, inplace=True)
df_works.drop(['id_doc','autores_cnpq','autores',
                'idioma','anais','natureza','ano'], axis=1, inplace=True)
df_works.dropna(inplace=True)
#df_works.info()

In [4]:
frames = [df_papers, df_works]

# Concatena Artigos e Trabalhos
df_lattes = pd.concat(frames)
df_lattes.sort_values(by=['id'], ignore_index=True, inplace=True)

del [frames, df_papers, df_works]
#df_lattes.info()

df_lattes = df_lattes.groupby('id', sort=False).titulo.apply(' '.join).reset_index(name='titulo')

In [5]:
def decode_html(text):
    decoded = html.unescape(text)
    return decoded

def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

def remove_separation(text):
    table = text.maketrans('-/', '  ')
    return text.translate(table)

def strip_ponct(text):
    table = str.maketrans({key: None for key in string.punctuation})
    return text.translate(table)

def one_space(text):
    return " ".join(text.split())

df_lattes['titulo'] = df_lattes['titulo'].apply(decode_html)
df_lattes['titulo'] = df_lattes['titulo'].apply(strip_html)
df_lattes['titulo'] = df_lattes['titulo'].apply(remove_separation)
df_lattes['titulo'] = df_lattes['titulo'].apply(strip_ponct)
df_lattes['titulo'] = df_lattes['titulo'].apply(one_space)

In [6]:
path = './data/corpus/'
Path(path).mkdir(exist_ok=True)
dev_file = '%scorpus.csv' % path
ida_file = '%sauthors.csv.xz' % path
pre_file = '%scorpus.csv.xz' % path

df_lattes.to_csv(dev_file, index=None, columns = ['titulo'])
df_lattes.to_csv(ida_file, index=None, compression='xz', columns = ['id'])
df_lattes.to_csv(pre_file, index=None, compression='xz', columns = ['titulo'])